In [1]:
#4. 인공 신경망(ANN-Artificial Neural Network)

In [3]:
#입력층
#: 입력 데이터를 받아들임
#출력층
#: 입력 데이터를 처리하고 특징을 추출
#은닉층
#: 최종 예측값을 출력

In [ ]:
#순전파

In [4]:
#손실함수
#: 오차를 균등하게 만들어주는 것
# -> |20-30| = |-10|
# -> |20-10| = |10|

In [6]:
#역전파
# f1(ax+b)
# => f2(f1(ax+b)+d

#함수가 여러 개 엮여있을 때 미분의 연쇄법칙을 사용하면 parameter들을 업데이트할 수 있음

#parameter : 업데이트하기 위한 가중치
#가중치에는 weight, bias

In [7]:
# 더이상 학습이 진행되지 않는 상황 -> 수렴했다.
#local optimum : 지역 최적값에 빠졌다
#global optimum:전역 최적값

#local optimum과 global optimum은 설정값에 따라 의존적
#=>hyper parameter라고 함
# 하지만 hyper parameter의 폭이 너무 커서 local optimum 조차도 못찾을 수도 있음.

In [8]:
#딥러닝을 통해 해결하고자 하는 문제를 정의하고
#데이터 수집
#처리
#딥러닝 모델 설계
#평가
#개선

In [9]:
#출력레이어의 유형과 활용

#회귀문제
#1차원

#이진 분류 문제
#확률이 나오게 됨

#다중 클래스 분류 모델

In [10]:
#실습: 간단한 인공 신경망 모델 구현 (PyTorch)

In [11]:
#MNIST 데이터셋 사용 : 숫자 이미지를 분류하는 모델 구현

In [1]:
import torch #핵심 라이브러리
import torch.nn as nn #뉴럴 네트워크 :신경망 구축을 위한 내용 포함
import torch.optim as optim #최적화 부분
import torchvision #이미지 처리를 위한 라이브러리
import torchvision.transforms as transforms #전처리를 위한 라이브러리

In [14]:
#데이터 로드
# 데이터셋 전처리
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# MNIST 데이터셋 로드
trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)

trainset

Failed to download (trying next):
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1000)>



100.0%


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1000)>



100.0%


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1000)>



100.0%


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1000)>



100.0%

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



Dataset MNIST
    Number of datapoints: 60000
    Root location: ./data
    Split: Train
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=(0.5,), std=(0.5,))
           )

In [15]:
trainloader

In [16]:
class SimpleANN(nn.Module):
    def __init__(self):
        super(SimpleANN, self).__init__()
        self.fc1 = nn.Linear(28 * 28, 128)  # 입력층에서 은닉층으로
        self.fc2 = nn.Linear(128, 64)       # 은닉층에서 은닉층으로
        self.fc3 = nn.Linear(64, 10)        # 은닉층에서 출력층으로

    def forward(self, x):
        x = x.view(-1, 28 * 28)  # 입력 이미지를 1차원 벡터로 변환
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [17]:
# 모델 초기화
model = SimpleANN()

correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct / total:.2f}%')


# 손실 함수와 최적화 알고리즘 정의
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

# 모델 학습
for epoch in range(10):  # 10 에포크 동안 학습
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data

        # 기울기 초기화
        optimizer.zero_grad()

        # 순전파 + 역전파 + 최적화
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # 손실 출력
        running_loss += loss.item()
        if i % 100 == 99:  # 매 100 미니배치마다 출력
            print(f'[Epoch {epoch + 1}, Batch {i + 1}] loss: {running_loss / 100:.3f}')
            running_loss = 0.0

print('Finished Training')

Accuracy of the network on the 10000 test images: 9.84%
[Epoch 1, Batch 100] loss: 1.375
[Epoch 1, Batch 200] loss: 0.492
[Epoch 1, Batch 300] loss: 0.393
[Epoch 1, Batch 400] loss: 0.354
[Epoch 1, Batch 500] loss: 0.320
[Epoch 1, Batch 600] loss: 0.296
[Epoch 1, Batch 700] loss: 0.292
[Epoch 1, Batch 800] loss: 0.273
[Epoch 1, Batch 900] loss: 0.240
[Epoch 2, Batch 100] loss: 0.234
[Epoch 2, Batch 200] loss: 0.193
[Epoch 2, Batch 300] loss: 0.209
[Epoch 2, Batch 400] loss: 0.174
[Epoch 2, Batch 500] loss: 0.182
[Epoch 2, Batch 600] loss: 0.186
[Epoch 2, Batch 700] loss: 0.187
[Epoch 2, Batch 800] loss: 0.148
[Epoch 2, Batch 900] loss: 0.152
[Epoch 3, Batch 100] loss: 0.140
[Epoch 3, Batch 200] loss: 0.142
[Epoch 3, Batch 300] loss: 0.142
[Epoch 3, Batch 400] loss: 0.134
[Epoch 3, Batch 500] loss: 0.141
[Epoch 3, Batch 600] loss: 0.124
[Epoch 3, Batch 700] loss: 0.137
[Epoch 3, Batch 800] loss: 0.115
[Epoch 3, Batch 900] loss: 0.129
[Epoch 4, Batch 100] loss: 0.108
[Epoch 4, Batch 200]

In [19]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        print(images, labels)

print(f'Accuracy of the network on the 10000 test images: {100 * correct / total:.2f}%')


tensor([[[[-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          ...,
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.]]],


        [[[-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          ...,
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.]]],


        [[[-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          ...,
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.]]],


        ...,


        [[[-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., 